In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.hash import hash_func_builder
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic
from JAxtar.hash import HashTable

/tmp/ipykernel_3692962/2078866209.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
2024-08-05 09:00:31.021593: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
count = 1000
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(2),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.build(puzzle.State, 1, int(1e5))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)

0.39784979820251465


In [4]:
batch = 4000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
for i in range(18):
    sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(i + 256),count))
    idx, table_idx, old_found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    batched_sample, filled = HashTable.make_batched(puzzle.State, sample, batch)
    start = time.time()
    table, inserted, _, _ = parallel_insert(table, batched_sample, filled)
    print(time.time()-start)
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    print(jnp.mean(found), jnp.mean(old_found), jnp.mean(inserted))

1.4837102890014648
1.0 0.0 0.25
1.433384656906128
0.9990001 0.0 0.25
0.000736236572265625
1.0 0.0 0.25
0.0006349086761474609
1.0 0.0 0.25
0.0006351470947265625
1.0 0.0 0.25
0.0005526542663574219
1.0 0.0 0.25
0.0005509853363037109
0.9990001 0.0 0.25
0.0005288124084472656
1.0 0.0 0.25
0.0005342960357666016
0.9990001 0.0 0.25
0.0004925727844238281
1.0 0.0 0.25
0.0004925727844238281
0.9990001 0.0 0.25
0.0004966259002685547
1.0 0.0 0.25
0.0004980564117431641
0.9990001 0.0 0.25
0.0004954338073730469
1.0 0.0 0.25
0.0005993843078613281
0.9990001 0.0 0.25
0.0004963874816894531
1.0 0.0 0.25
0.0005118846893310547
1.0 0.0 0.25
0.0004782676696777344
1.0 0.0 0.25


In [5]:
jnp.max(table.table_idx)

Array(2, dtype=uint8)

In [6]:
count = int(1e6)
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(2),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.build(puzzle.State, 1, int(1e7))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)

0.37995052337646484


In [7]:
batch = 10000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
for i in range(18):
    inserteds = []
    sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(i + 256),count))
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    same_ratio = jnp.mean(found)
    for j in trange(0, count, batch):
        table, inserted, _, _ = parallel_insert(table, sample[j:j+batch], jnp.ones(batch, dtype=jnp.bool_))
        inserteds.append(inserted)
    inserteds = jnp.concatenate(inserteds)
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    print(jnp.mean(found), same_ratio, jnp.mean(inserteds))

100%|██████████| 100/100 [00:03<00:00, 28.22it/s]


0.999972 0.0 1.0


100%|██████████| 100/100 [00:00<00:00, 634.48it/s]


0.999923 0.0 1.0


100%|██████████| 100/100 [00:00<00:00, 630.93it/s]


...

0.999511 0.0 1.0


100%|██████████| 100/100 [00:00<00:00, 429.93it/s]


0.999442 0.0 0.999999


 15%|█▌        | 15/100 [00:00<00:00, 354.15it/s]


KeyboardInterrupt: 

In [ ]:
jnp.max(table.table_idx)

Array(2, dtype=uint8)

In [ ]:
another_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(123),count))
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print(jnp.sum(found) / count)

1e-06


In [ ]:
start = time.time()
inserteds = []
for i in trange(count // batch):
    table, inserted, _, _ = parallel_insert(table, another_sample[i*batch:(i+1)*batch], jnp.ones(batch, dtype=jnp.bool_))
    inserteds.append(inserted)
print(jnp.sum(jnp.concatenate(inserteds)) / count)
print("insert time:", time.time()-start)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, new_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)

100%|██████████| 100/100 [00:01<00:00, 66.03it/s]


0.999999
insert time: 1.5196442604064941
check time: 0.13991403579711914
0.996594
check time: 0.15191936492919922
1e-06
check time: 0.3145577907562256
0.993682
